<a href="https://colab.research.google.com/github/jansoe/BootstrapPrediction/blob/master/A6_1_jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Wortvektoren - Teil 2

## 6.1 Word2Vec selbst trainieren

Nachdem wir uns im letzten Teil mit bereits trainierten Wortvektoren beschäftigt haben, zeigen wir Ihnen hier, wie Sie ein Word2Vec-Modell anhand eines Textdatensatzes trainieren können.

Das Verfahren ist wie gehabt:
- Erstellen Sie eine Kopie dieses Notebooks in ihrem Google Drive (vorgeschlagene Umbenennung: "A6_1 - Vorname, Nachname")
- Editieren Sie die Text- und Codezellen.
- Schicken Sie uns einen Freigabelink zum Kommentieren Ihres Notebooks an

### Vorbereitungen

Zunächst das Übliche: Verwendete Bibliotheken importieren und Warnungen ausschalten.

In [ ]:
import matplotlib.pyplot as plt
import nltk
import gensim
from pprint import pprint
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Als Datensatz verwenden wir die Wikipedia-Artikel, die wir bereits in Abschnitt 5 heruntergeladen und vorbereitet haben. Entsprechend müssen wir unser Google Drive einbinden.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Ersetzen Sie in der folgenden Code-Zelle die Pfadangabe mit dem enstprechenden Verzeichnis Ihres Google Drives.

In [ ]:
path = '/content/drive/My Drive/KI-Schule/2 - Textdaten/wiki'
!ls "{path}"

ls: cannot access '/content/drive/My Drive/KI-Schule/2 - Textdaten/wiki': No such file or directory


Jetzt sollten die Wikipedia-Artikel aus A5_1 geladen werden können.


In [ ]:
import json
corpus = json.load(open(path + '/article_dict.json'))

Jetzt müssen wir die Rohdaten noch in eine für das Training des Modells geeignete Form bringen. Hierfür definieren wir uns wieder den bereits bekannten Tokenizer ...

In [ ]:
def tokenizer(text):
    tokens = list(gensim.utils.simple_tokenize(text))
    return tokens

... und definieren uns eine Funktion zum Rausfiltern von Stopwords auf Basis einer zuvor heruntergelandenen Liste.

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('german')
def is_not_stopword(word):
    return word not in stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Im nächsten Schritt können wir unsere Funktionen `tokenizer()` und `is_not_stopword()` anwenden, um uns eine vorverarbeitete Dokumentensammlung zu erstellen. Dieser Schritt kann einige Zeit in Anspruch nehmen.

In [ ]:
n_articles =  15000 # 15000 Artikel ermöglichen recht hohe Qualität; noch mehr dauert entsprechend länger

docs = []
for n_article, text in enumerate(list(corpus.values())[:n_articles]):
    tokens = tokenizer(text.lower())
    tokens = list(filter(is_not_stopword, tokens))
    if len(tokens) > 5:
        docs.append(tokens)
    if n_article % 100 == 0:
        print('\r', n_articles - n_article, 'articles still need processing', end='')
print('fertig!')

fertig!


### Ein Word2Vec-Modell trainieren

Anhand dieser Zusammenstellung von tokenisierten Dokumenten können wir nun ein Word2Vec-Modell trainieren. Auch dieser Schritt benötigt einige Minuten - leider ohne jegliche Form von Fortschrittsanzeige.

In [ ]:
my_model = gensim.models.Word2Vec(
    sentences = docs,
    window=15,
    size=150,
    min_count=50
)

Sie können anschließend das Modell in ihrem Google Drive abspeichern, damit Sie das Training nur einmal absolvieren müssen.

In [ ]:
my_model.save(path + "/my_word2vec.model")

Bei Bedarf können Sie das abgespeicherte Modell zu einem späteren Zeitpunkt wieder laden.

---
**Anmerkungen**

Auch wenn wir im Rahmen dieses Notebooks hiervon nicht Gebrauch machen, könnte das Modell über die Methode train() immer wieder mit weiteren Texten trainiert werden - allerdings muss es sich dabei um Texte des gleichen Korpus handeln, da sich das Vokabular (und damit die Länge der Input-Vektoren) bei dieser Art von Modellen im Nachhinein nicht mehr erweitert werden kann:

`my_model.train(new_corpus, total_examples=1, epochs=1)`

In [ ]:
my_model = gensim.models.Word2Vec.load(path + "/my_word2vec.model")

Wir können das Modell kurz testen, indem wir uns z.B. die fünf Wörter ausgeben lassen, die dem Begriff "Intelligenz" am ähnlichsten sind (auf Basis der Kosinusähnlichkeit der zugehörigen Wordvektoren).

In [ ]:
my_model.wv.most_similar(positive=['intelligenz'], topn=5)

[('robotern', 0.7771303057670593),
 ('neuronale', 0.7405957579612732),
 ('artificial', 0.7331240177154541),
 ('kognitive', 0.7283046841621399),
 ('künstliche', 0.7177648544311523)]

### 6.1.0 Grenzen des Modells

Können Sie ein Wort finden, dessen fünf ähnlichste Begriffe Sie nicht überzeugen?

### Visualisierung ähnlicher Wortvektoren

In [ ]:
top_similarity = my_model.wv.most_similar(positive=['intelligenz'], topn=1000)
selected_words = [word for word, similartiy in top_similarity]
vectors = [my_model.wv.word_vec(word, use_norm=True) for word in selected_words]

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE

tsne = TSNE(
    n_components=2,
    perplexity = 30
)
vectors_2D = tsne.fit_transform(vectors)

to_plot = pd.DataFrame(vectors_2D, columns=['x', 'y'])
to_plot['labels'] = selected_words

px.scatter(to_plot, x='x', y='y', hover_name='labels')

![insitubytes](https://drive.google.com/uc?id=1EAJK7AI9tcZRo3VvYq7vEKGxk7vmK2Ff)